In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, MetaData, Table, text

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

In [3]:
USER_DB = POSTGRES_UTEA['USER']
PASS_DB = POSTGRES_UTEA['PASSWORD']
HOST_DB = POSTGRES_UTEA['HOST']
PORT_DB = POSTGRES_UTEA['PORT']
NAME_DB = POSTGRES_UTEA['DATABASE']

ENGINE = create_engine(f'postgresql+psycopg://{USER_DB}:{PASS_DB}@{HOST_DB}:{PORT_DB}/{NAME_DB}')

PATH_GRUPOS_COSECHA = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPO_COSECHA.xlsx'
PATH_PLAN_ZAFRA = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\COMPROMISOS ZAFRA 2025 - Documents\PLAN ZAFRA 2025\PLAN DE ZAFRA OFICIAL.xlsx'
PATH_LAICA = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\COOR_GERENCIA_CANA - PARTES_LAICA\LAICA 6_2025.xlsx'
PATH_GRUPOS_ECO = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPOS ECONOMICOS.xlsx'
PATH_COMPROMISO_FIRMADO = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\COMPROMISOS FIRMADOS ZAFRA.xlsx'

metadata = MetaData()

In [14]:
def cargar_codigos_cosecha():
    try:
        df = pd.read_excel(PATH_GRUPOS_COSECHA, sheet_name='2025')
        df = df[['INS', 'CODIGO CAÑERO', 'NOMBRE CAÑERO', 'GRUPO DE COSECHA', 'DELEGADO']]
        df = df.rename(columns={
            'INS': 'inst',
            'CODIGO CAÑERO': 'cod_ca',
            'NOMBRE CAÑERO': 'nom_ca',
            'GRUPO DE COSECHA': 'cod_cos',
            'DELEGADO': 'delegado'
        })
        df = df.sort_values(by=['cod_cos', 'inst', 'cod_ca'])
        return df
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {e}")
        return pd.DataFrame()

def cargar_cod_cos_a_db(df):
    # Convertir tipos para Postgres
    df['inst']     = df['inst'].astype('Int64')
    df['cod_ca']   = df['cod_ca'].astype('Int64')
    df['cod_cos']  = df['cod_cos'].astype('Int64')
    df['nom_ca']   = df['nom_ca'].astype(str)

    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_codigos_cosecha RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_codigos_cosecha',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de codigos cosecha")
    
def cargar_plan_zafra():
    try:
        df = pd.read_excel(PATH_PLAN_ZAFRA, sheet_name='BASE CAÑEROS')
        df = df[['INST.', 'COD. CAÑERO.1', 'NOMBRE DEL CAÑERO', 'TONELADAS COMPROMISO']]        
        df = df.rename(columns={
            'INST.': 'inst',
            'COD. CAÑERO.1': 'cod_ca',
            'NOMBRE DEL CAÑERO': 'nom_ca',
            'TONELADAS COMPROMISO': 'compromiso'
        })
        df = df.sort_values(by=['inst', 'cod_ca'])
        return df
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {e}")
        return pd.DataFrame()

def cargar_plan_zafra_a_db(df):
    # Convertir tipos para Postgres
    df['inst']     = df['inst'].astype('Int64')
    df['cod_ca']   = df['cod_ca'].astype('Int64')
    df['nom_ca']   = df['nom_ca'].astype(str)
    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_plan_zafra RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_plan_zafra',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de plan de zafra")
    
def get_resumen_estimativa():
    try:
        query = "select unidad_01, unidad_03, categoria, sum(area) as area from catastro_iag.estimativa group by unidad_01, unidad_03, categoria"
        df = pd.read_sql(query, ENGINE)
        return df
    except Exception as e:
        print("❌ Error al leer datos de ESTIMATIVA:", e)
        return pd.DataFrame()
    return None

def cargar_resumen_estimativa_a_db(df):
    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_resumen_estimativa RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_resumen_estimativa',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de resumen estimativa")
    
def cargar_laica():
    try:
        df = pd.read_excel(PATH_LAICA, sheet_name='Datos')
        df = df[['Fecha', 'Cod. Cañero', 'Nombre Cañero', 'kg Caña Neta']]        
        df = df.rename(columns={
            'Fecha': 'fecha',
            'Cod. Cañero': 'cod_ca',
            'Nombre Cañero': 'nom_ca',
            'kg Caña Neta': 'kg_entregado'
        })
        return df
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {e}")
        return pd.DataFrame()

def cargar_datos_laica_a_db(df):
    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_laica RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_laica',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de datos laica")


def cargar_grupos_eco():
    try:
        df = pd.read_excel(PATH_GRUPOS_ECO, sheet_name='RESUMEN_GRUPO_ECO')
        df = df.rename(columns={
            'DEUDOR': 'deudor',
            'CODIGO': 'codigo',
            'NOMBRE': 'nombre',
            'GRUPO ECONOMICO': 'grupo_economico',
            'COD. INSTITUCION': 'cod_institucion',
            'ORDEN': 'orden',
            'Cabeza del Grupo': 'cabeza_del_grupo'
        })
        return df
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {e}")
        return pd.DataFrame()

def cargar_datos_grupo_eco_a_db(df):
    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_grupo_eco RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_grupo_eco',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de datos grupos economicos")



def cargar_compromiso_firmado():
    try:
        df = pd.read_excel(PATH_COMPROMISO_FIRMADO, sheet_name='RESUMEN_COMPROMISO')
        df = df.rename(columns={
            'COD.': 'cod',
            'TONELADAS': 'toneladas',
            'GRUPO_ECO': 'grupo_eco'
        })
        return df
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {e}")
        return pd.DataFrame()

def cargar_compromiso_firmado_a_db(df):
    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_compromiso_firmado RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_compromiso_firmado',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de datos grupos economicos")

In [5]:
codigos_cosecha = cargar_codigos_cosecha()
cargar_cod_cos_a_db(codigos_cosecha)

✅ Se ha actualziado la tabla de codigos cosecha


In [6]:
plan_zafra = cargar_plan_zafra()
cargar_plan_zafra_a_db(plan_zafra)

✅ Se ha actualziado la tabla de plan de zafra


In [7]:
resumen_estimativa = get_resumen_estimativa()
cargar_resumen_estimativa_a_db(resumen_estimativa)

✅ Se ha actualziado la tabla de resumen estimativa


In [11]:
datos_laica = cargar_laica()
datos_laica['tn_entregado'] = datos_laica['kg_entregado'] / 1000
cargar_datos_laica_a_db(datos_laica)

✅ Se ha actualziado la tabla de datos laica


In [12]:
datos_grupo_eco = cargar_grupos_eco()
cargar_datos_grupo_eco_a_db(datos_grupo_eco)

✅ Se ha actualziado la tabla de datos grupos economicos


In [15]:
datos_compromiso_firmado = cargar_compromiso_firmado()
cargar_compromiso_firmado_a_db(datos_compromiso_firmado)

✅ Se ha actualziado la tabla de datos grupos economicos
